In [ ]:
%run ./01v_volve_generate_data.ipynb

import tensorflow as tf
import keras
from keras import layers
from keras import backend as K
#import src

import scipy.stats as stats
import math

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)

num_features = 1
num_hidden_units1 = 120

num_hidden_units2 = 150
num_responses = 1
max_epochs = 500
mini_batch_size = 5

lstm_net = keras.models.Sequential([
    layers.Bidirectional(
        layers.LSTM(units=num_hidden_units1, return_sequences=True
    ), input_shape=(160,1)),
    layers.BatchNormalization(epsilon=1e-4),
    layers.ReLU(),
    layers.Dense(150),
    layers.Dense(50),
    layers.Bidirectional(
        layers.LSTM(units=num_hidden_units2, return_sequences=True)
    ),
    layers.Dense(num_responses)
])

lstm_net.summary()

In [ ]:
def scheduler(epoch, lr):
    if epoch % 150 == 0 and epoch != 0:
        value = lr * tf.math.exp(-0.1)
        K.set_value(lstm_net.optimizer.lr, value)
        return K.get_value(lstm_net.optimizer.lr)
    else:
        return K.get_value(lstm_net.optimizer.lr)
    
callback =  tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                             mode='min',
                                             restore_best_weights=True,
                                             verbose=2,
                                             patience=300)
    
lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

optimizer = keras.optimizers.Adam(learning_rate=0.001, clipnorm=2)

loss_metrics = [
    'mse',
    'mae'
]

metric_funcs = [
    tf.keras.metrics.MeanSquaredError(),
    tf.keras.metrics.MeanAbsoluteError(),
    tf.keras.metrics.RootMeanSquaredError()
]

lstm_net.compile(optimizer=optimizer,
                 loss=loss_metrics,
                 metrics=metric_funcs)

history_lstm = lstm_net.fit(trainX,
                            trainIp,
                            shuffle=False,
                            batch_size=mini_batch_size,
                            epochs=max_epochs,
             #               callbacks=[lr_scheduler],
                            callbacks=callback,
                            validation_data=(x_valid, y_valid),
                            validation_freq=1)

In [ ]:
y_predicted = lstm_net.predict(x_test, verbose=1)

n = 3
plt.figure(figsize=(20, 10))
for i in range(1, n + 1):
    ax = plt.subplot(2, n, i)
    ax.set_title("y_validation {0}".format(i-1))
    plt.plot(y_test[i-1, :, 0])
    plt.plot(y_predicted[i-1, :, 0], '--r')
    plt.legend(['original Ip', 'predicted Ip'])

#     ax = plt.subplot(2, n, i + n, sharey=ax)
#     ax.set_title("y_predicted {0}".format(i-1))
#     plt.plot(y_predicted[i-1, :, 0])
plt.show()

In [ ]:
# ypredicted_f32 = tf.cast(y_predicted, dtype=tf.float32)
# ytest_f32 = tf.cast(y_test, dtype=tf.float32)

# result = tf.image.ssim(ytest_f32, ypredicted_f32, max_val=1)

# print(result)
predict_image = y_predicted.flatten()
predict_image = (predict_image-np.min(predict_image))/(np.max(predict_image)-np.min(predict_image))
predict_image = predict_image.reshape(y_predicted.shape)
predict_image = (predict_image).astype('float32')

target_image = y_test.flatten()
target_image = (target_image-np.min(target_image))/(np.max(target_image)-np.min(target_image))
target_image = target_image.reshape(y_test.shape)
target_image = target_image.astype('float32')

SSIM = tf.image.ssim(target_image, predict_image, max_val=1).numpy()
print('SSIM: ', SSIM)

fig1, axes1 = plt.subplots(nrows=1, ncols=2)
axes1[0].imshow(predict_image)
axes1[0].set_title("Predicted")
axes1[1].imshow(target_image)
axes1[1].set_title("Target")